In [ ]:
import numpy as np

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

In [ ]:
from astropy.io import fits
from astropy.cosmology import Planck15 as cosmo

In [ ]:
# import forward test module
import fwtest

In [ ]:
# load `forward' magnitudes module
from forward import mag

In [ ]:
# the SEDs we want to use
sed_fits = fits.open('https://github.com/blanton144/kcorrect/raw/master/data/templates/k_nmf_derived.default.fits')

In [ ]:
# the filters for our instrument
filt_fits = fits.open('http://www.ctio.noao.edu/noao/sites/default/files/DECam/STD_BANDPASSES_DR1.fits')

In [ ]:
# emitted wavelength lambda and SED function f
le = sed_fits[11].data
fe = sed_fits[1].data

In [ ]:
# make sure shapes are ok
np.shape(le), np.shape(fe)

In [ ]:
# number of SEDs
nsed = len(fe)

In [ ]:
# filter bands to use
bands = 'griz'

In [ ]:
# number of filters
nfilt = len(bands)

In [ ]:
# get the filter columns
lx = filt_fits[1].data['LAMBDA']
Rx = [filt_fits[1].data[b] for b in bands]

In [ ]:
# colours for plotting
plotcolours = ['green', 'red', 'brown', 'purple']

In [ ]:
# show what the filter looks like
for i in range(nfilt):
    plt.plot(lx, Rx[i], color=plotcolours[i], label=bands[i])
plt.xlabel(r'$\lambda$')
plt.ylabel(r'$R_x(\lambda)$')
plt.legend()
plt.show()

In [ ]:
# redshift range
z = np.arange(0, 4, 0.01)

In [ ]:
# compute AB zeropoints for filters
m0 = mag.abzeropt(lx, Rx)

In [ ]:
# zeropoints should be about 2
m0

In [ ]:
# compute fluxes
f = mag.flux(z, le, fe, lx, Rx)

In [ ]:
# the shape should be (len(z), len(fe), len(Rx))
np.shape(f)

In [ ]:
# convert fluxes to AB magnitudes
m = -2.5*np.log10(f) - m0

In [ ]:
# plot the AB magnitudes
plt.figure(figsize=(8, 6))
for i in range(nsed):
    plt.subplot(3, 3, i+1)
    plt.title(r'SED {}'.format(i+1))
    plt.xlabel(r'$z$')
    plt.ylabel(r'${\rm mag}_x$')
    for j in range(nfilt):
        plt.plot(z, m[:,i,j], color=plotcolours[j], label=bands[j])
    plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
# plot the colours
nc = nfilt-1
fig, axes = plt.subplots(nsed, nc, figsize=(8, 6), sharex='all', sharey='all')
for i in range(nsed):
    for j in range(nc):
        ax = axes[i,j]
        if i == nsed-1:
            ax.set_xlabel(r'$z$')
        ax.set_ylabel(r'${}-{}$'.format(bands[j], bands[j+1]))
        ax.plot(z, c[:,i,j]-c[:,i,j+1])
plt.tight_layout()
plt.show()